In [11]:
#hedhy mch khayba metaa model
from kafka import KafkaConsumer
from influxdb_client import InfluxDBClient, Point, WritePrecision
from json import loads
import numpy as np
from tabulate import tabulate
import joblib
import time
from influxdb_client.client.write_api import SYNCHRONOUS

tp_fp_fn_counts = {"TP": 0, "FP": 0, "FN": 0, "TN": 0}


# threshold values for blood pressure levels
normal_sbp_upper = 120
normal_dbp_upper = 80
elevated_sbp_upper = 129
elevated_dbp_upper = 89
hypertension_stage_1_sbp_upper = 139
total_data_processed =0
hypertension_stage_1_dbp_upper = 89
 
def categorize_bp(sbp, dbp, hr, spo2):
    if sbp == 0 and dbp == 0 and (hr == 0 or hr > 80) and (spo2 == 0 or spo2 > 80):
        return 'Invalid Measurement'  
    elif sbp < normal_sbp_upper and dbp < normal_dbp_upper:
        return 'Normal'
    elif sbp < elevated_sbp_upper and dbp < elevated_dbp_upper:
        return 'Elevated'
    elif sbp < hypertension_stage_1_sbp_upper and dbp < hypertension_stage_1_dbp_upper:
        return 'Hypertension Stage 1'
    else:
        return 'Hypertension Stage 2 or higher'
 
 
# Kafka broker address
bootstrap_servers = 'localhost:9093'

# Kafka topic to consume messages from
kafka_topic = 'topicI'

# Consumer group ID
group_id = 'python-consumer'

# InfluxDB connection details
influxdb_host = 'localhost'
influxdb_port = 8086
influxdb_username = 'Rahma'
influxdb_password = 'rahma123'
bucket = "BloodPressure"
organization = "OstProject"
influxdb_token = "RTTxO5B8wP9KiEoh0igojV_fq4OLNRMBuvljnJdmbQXXUSzZy5g_s8rfWS4Lu7ZmDHzJ0CO0yALo8hg2vKVFnw=="

# Specify the path to the offline model.pkl file
model_path = 'OC_SVM.pkl'

# Load the trained model
with open(model_path, 'rb') as model_file:
    clf = joblib.load(model_file)

# Create InfluxDB client
client = InfluxDBClient(url=f"http://{influxdb_host}:{influxdb_port}", token=influxdb_token, org=organization)

# Create Kafka consumer
consumer = KafkaConsumer(
    kafka_topic,
    bootstrap_servers=['localhost:9093'],
    auto_offset_reset='earliest',
    api_version=(0, 10),
    enable_auto_commit=True,
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

print("Connection established")

# Real-time anomaly detection loop
while True:
    try:
        # Poll for new messages
        for tp, records in consumer.poll(timeout_ms=1000).items():
            for record in records:
                # Extract features from the incoming data
                data = record.value
                sbp_bp = float(data.get('SBP'))
                dbp_bp = float(data.get('DBP'))
                pp= float(data.get('PP'))
                co = float(data.get('CO'))
                temps=data.get('Timestamp')
                # Update total data processed count
                total_data_processed =total_data_processed + 1

                # Add a check for None before converting to integer
                actual_anomaly = 0
                predicted_anomaly=0
                                    
                
                # Use the trained model to get anomaly scores
                features = np.array([[sbp_bp, dbp_bp]])

                # Ensure model prediction works without errors
                anomaly_score = clf.decision_function(features)[0]

                if -0.00035 > anomaly_score > -0.00038 or anomaly_score < -45:
                    predicted_anomaly = 1
                else:
                    predicted_anomaly = 0

                if sbp_bp == 0 or dbp_bp == 0 :
                    actual_anomaly =1
                elif sbp_bp > 120 or dbp_bp > 80:
                    actual_anomaly =1
                elif sbp_bp < 120 and dbp_bp < 80:
                    actual_anomaly =0
                else:
                    actual_anomaly =1

                # Update TP, FP, FN counts
                if predicted_anomaly == 1 and actual_anomaly == 1:
                    tp_fp_fn_counts["TP"] += 1
                elif predicted_anomaly == 1 and actual_anomaly == 0:
                    tp_fp_fn_counts["FP"] += 1
                elif predicted_anomaly == 0 and actual_anomaly == 1:
                    tp_fp_fn_counts["FN"] += 1
                elif predicted_anomaly == 0 and actual_anomaly == 0:
                    tp_fp_fn_counts["TN"] += 1

                # Create InfluxDB point
                point = Point("kafera") \
                .field("SBP", sbp_bp) \
                .field("DBP", dbp_bp) \
                .field("CO", co) \
                .field("PP", pp) \
                .field("AnomalyScore", float(anomaly_score)) \
                .field("Predicted_anomaly", int(predicted_anomaly)) \
                .field("Actual_anomaly",int(actual_anomaly)) \
                .field("TP", tp_fp_fn_counts["TP"]) \
                .field("FP", tp_fp_fn_counts["FP"]) \
                .field("FN", tp_fp_fn_counts["FN"]) \
                .field("TN", tp_fp_fn_counts["TN"])\
                .field("TotalDataProcessed", total_data_processed)\


                # Write the point to InfluxDB
                write_api = client.write_api(write_options=SYNCHRONOUS)
                write_api.write(bucket=bucket, org=organization, record=point)

               # Print the table
                table = [
                        ["SBP", "DBP", "Anomaly Score", "predicted_anomaly", "actual_anomaly", "TP", "FP", "FN",'TIME'],
                        [sbp_bp, dbp_bp, anomaly_score, predicted_anomaly, actual_anomaly,
                         tp_fp_fn_counts["TP"], tp_fp_fn_counts["FP"], tp_fp_fn_counts["FN"],temps]
                ]
                print(tabulate(table, headers="firstrow", tablefmt="grid"))
                time.sleep(2)

    except Exception as e:
        print(f"Error processing message: {e}")


Connection established
+-------+-------+-----------------+---------------------+------------------+------+------+------+-----------------+
|   SBP |   DBP |   Anomaly Score |   predicted_anomaly |   actual_anomaly |   TP |   FP |   FN | TIME            |
+=======+=======+=================+=====================+==================+======+======+======+=================+
|  95.7 |  48.4 |         -32.784 |                   0 |                0 |    0 |    0 |    0 | 2020-10-25 0:13 |
+-------+-------+-----------------+---------------------+------------------+------+------+------+-----------------+
+-------+-------+-----------------+---------------------+------------------+------+------+------+-----------------+
|   SBP |   DBP |   Anomaly Score |   predicted_anomaly |   actual_anomaly |   TP |   FP |   FN | TIME            |
+=======+=======+=================+=====================+==================+======+======+======+=================+
|  94.5 |  47.7 |        -34.5049 |              

KeyboardInterrupt: 